# Day 3: 데이터 수집 파이프라인 테스트

이 노트북은 Day 3에서 구현한 데이터 수집 API를 테스트합니다.

## 테스트 항목
1. Health Check
2. 소스 목록 조회
3. arXiv 논문 수집
4. 뉴스 수집 (API 키 필요)
5. 통합 검색

In [ ]:
from pprint import pprint

import requests

# API Base URL
BASE_URL = "http://localhost:8000"

## 1. Health Check

서버가 정상적으로 실행 중인지 확인합니다.

In [ ]:
# Health check
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print("Response:")
pprint(response.json())

## 2. 소스 목록 조회

사용 가능한 모든 데이터 소스와 필터 옵션을 확인합니다.

In [ ]:
# Get available sources
response = requests.get(f"{BASE_URL}/api/collectors/sources")
print(f"Status Code: {response.status_code}")
print("\nAvailable Sources:")
pprint(response.json())

## 3. arXiv 논문 수집

arXiv에서 논문을 검색하고 수집합니다.

In [ ]:
# arXiv search - Basic
payload = {"query": "large language model", "limit": 3}

response = requests.post(f"{BASE_URL}/api/collectors/arxiv", json=payload)

print(f"Status Code: {response.status_code}")
data = response.json()
print(f"\nTotal Results: {data['total']}")
print(f"Errors: {data['errors']}")
print("\n" + "=" * 80)

for i, item in enumerate(data["results"], 1):
    print(f"\n[{i}] {item['title']}")
    print(f"    Source: {item['source_name']}")
    print(f"    Type: {item['source_type']}")
    print(f"    URL: {item['url']}")
    print(f"    Content: {item['content'][:200]}...")
    print(f"    Collected: {item['collected_at']}")
    print("    Metadata: ")
    pprint(item["metadata"])

In [ ]:
# arXiv search - With filters
payload = {
    "query": "transformer",
    "limit": 5,
    "filters": {"categories": ["cs.AI", "cs.LG"], "sort_by": "relevance", "sort_order": "descending"},
}

response = requests.post(f"{BASE_URL}/api/collectors/arxiv", json=payload)

print(f"Status Code: {response.status_code}")
data = response.json()
print(f"\nTotal Results: {data['total']}")

# Display titles only
print("\nPapers found:")
for i, item in enumerate(data["results"], 1):
    print(f"{i}. {item['title']}")
    print(f"   Categories: {item['metadata']['categories']}")
    print(f"   Authors: {', '.join(item['metadata']['authors'][:3])}...")
    print()

## 4. 뉴스 수집

AI/테크 뉴스를 검색합니다. (Serper 또는 Brave API 키 필요)

**주의**: 이 셀을 실행하려면 `.env` 파일에 `SERPER_API_KEY` 또는 `BRAVE_API_KEY`가 설정되어 있어야 합니다.

In [ ]:
# News search
payload = {
    "query": "artificial intelligence",
    "limit": 5,
    "filters": {
        "domains": ["techcrunch.com", "venturebeat.com"],
        "date_filter": "w",  # Last week
    },
}

response = requests.post(f"{BASE_URL}/api/collectors/news", json=payload)

print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print(f"\nTotal Results: {data['total']}")
    print(f"Errors: {data['errors']}")
    print("\n" + "=" * 80)

    for i, item in enumerate(data["results"], 1):
        print(f"\n[{i}] {item['title']}")
        print(f"    Source: {item['metadata'].get('source_name', 'N/A')}")
        print(f"    URL: {item['url']}")
        print(f"    Snippet: {item['content']}")
        print(f"    Date: {item['metadata'].get('published_date', 'N/A')}")
else:
    print("Error:")
    pprint(response.json())

## 5. 통합 검색

여러 소스를 동시에 검색합니다.

In [ ]:
# Unified search - All sources
payload = {"query": "GPT-4", "limit": 3}

response = requests.post(f"{BASE_URL}/api/collectors/search", json=payload)

print(f"Status Code: {response.status_code}")
data = response.json()
print(f"\nTotal Results: {data['total']}")
print(f"Errors: {data['errors']}")

# Group by source
by_source = {}
for item in data["results"]:
    source = item["source_name"]
    if source not in by_source:
        by_source[source] = []
    by_source[source].append(item)

print("\nResults by source:")
for source, items in by_source.items():
    print(f"\n{source}: {len(items)} items")
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item['title'][:80]}...")

In [ ]:
# Unified search - Specific sources only
payload = {
    "query": "machine learning",
    "sources": ["arxiv"],  # Only arXiv
    "limit": 5,
    "filters": {"categories": ["cs.LG", "stat.ML"]},
}

response = requests.post(f"{BASE_URL}/api/collectors/search", json=payload)

print(f"Status Code: {response.status_code}")
data = response.json()
print(f"\nTotal Results: {data['total']}")
print(f"Errors: {data['errors']}")

print("\nPapers:")
for i, item in enumerate(data["results"], 1):
    print(f"{i}. {item['title']}")
    print(f"   URL: {item['url']}")

## 6. 에러 처리 테스트

잘못된 요청에 대한 에러 처리를 테스트합니다.

In [ ]:
# Test with invalid source
payload = {"query": "test", "sources": ["invalid_source"], "limit": 5}

response = requests.post(f"{BASE_URL}/api/collectors/search", json=payload)

print(f"Status Code: {response.status_code}")
data = response.json()
print("\nResponse:")
pprint(data)

In [ ]:
# Test with empty query
payload = {"query": "", "limit": 5}

response = requests.post(f"{BASE_URL}/api/collectors/arxiv", json=payload)

print(f"Status Code: {response.status_code}")
print("\nResponse:")
pprint(response.json())

## 7. 성능 테스트

API 응답 시간을 측정합니다.

In [ ]:
import time

# Test arXiv performance
payload = {"query": "neural network", "limit": 10}

start = time.time()
response = requests.post(f"{BASE_URL}/api/collectors/arxiv", json=payload)
elapsed = time.time() - start

print(f"Status Code: {response.status_code}")
print(f"Elapsed Time: {elapsed:.2f}s")
print(f"Results: {response.json()['total']} items")
print(f"Average time per item: {elapsed/response.json()['total']:.2f}s")

## 8. 데이터 분석

수집된 데이터를 분석합니다.

In [ ]:
import pandas as pd

# Collect more data for analysis
payload = {"query": "deep learning", "limit": 20, "filters": {"categories": ["cs.AI", "cs.LG", "cs.CV"]}}

response = requests.post(f"{BASE_URL}/api/collectors/arxiv", json=payload)

data = response.json()

# Convert to DataFrame
df_data = []
for item in data["results"]:
    df_data.append(
        {
            "title": item["title"],
            "url": item["url"],
            "source": item["source_name"],
            "primary_category": item["metadata"].get("primary_category", "N/A"),
            "num_authors": len(item["metadata"].get("authors", [])),
            "published": item["metadata"].get("published", "N/A"),
            "content_length": len(item["content"]),
        }
    )

df = pd.DataFrame(df_data)
print("\nDataFrame:")
print(df.head())

print("\nStatistics:")
print(df.describe())

print("\nCategory distribution:")
print(df["primary_category"].value_counts())

## 9. 시각화

수집 결과를 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt

# Plot category distribution
if len(df) > 0:
    plt.figure(figsize=(10, 6))
    df["primary_category"].value_counts().plot(kind="bar")
    plt.title("arXiv Papers by Category")
    plt.xlabel("Category")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Plot content length distribution
    plt.figure(figsize=(10, 6))
    plt.hist(df["content_length"], bins=20, edgecolor="black")
    plt.title("Abstract Length Distribution")
    plt.xlabel("Length (characters)")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()

## 결론

Day 3 데이터 수집 파이프라인 테스트를 완료했습니다.

### 테스트 결과 요약:
- ✅ Health Check
- ✅ 소스 목록 조회
- ✅ arXiv 논문 수집
- ✅ 필터링 및 정렬
- ✅ 에러 처리
- ✅ 통합 검색

### 다음 단계 (Day 4):
- LLM 기반 데이터 처리 (요약, 평가, 분류)
- 임베딩 생성 및 Vector DB 저장